<a href="https://colab.research.google.com/github/Gus-1003/Antares/blob/main/antares_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Videos:

In [ ]:
def remove_rats(images):
  """
  Removes noise from a set of images by computing the median image.

  Args:
      images (list): A list of input images.

  Returns:
      numpy.ndarray: The median image, after noise removal.
  """
  import numpy as np

  # Convert each image in the list to a NumPy array
  images_array = [np.array(image) for image in images]

  # Compute the median image along the specified axis
  median_image = np.median(images_array, axis=0)

  # Convert the median image data type to unsigned 8-bit integer
  median_image = median_image.astype('uint8')

  return median_image

In [ ]:
def extract_frames(video_package, image_output):
    """
    Extracts frames from videos in a video package and saves them as images.

    Args:
        video_package (list): A list of video file paths.
        image_output (str): The directory where extracted images will be saved.

    Returns:
        None
    """
    import os
    import cv2
    import numpy as np


    for video in video_package:
        if not os.path.exists(video):
            print(f"The video '{video}' doesn't exist.")
            return

        vidcap = cv2.VideoCapture(video)

        if not vidcap.isOpened():
            print(f"Failed to open the video '{video}'.")
            return

        images = []
        success, image = vidcap.read()
        cont = 0
        interval_between_frames = 15
        number_of_frames = 10
        file_name = os.path.basename(video).split('.')[0]

        while success and cont < number_of_frames:
            success, image = vidcap.read()
            if image is None:
                break

            images.append(image)
            vidcap.set(cv2.CAP_PROP_POS_FRAMES, int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)) + interval_between_frames * vidcap.get(cv2.CAP_PROP_FPS))
            cont += 1

        # Call the remove_rats function to process the extracted frames
        result = remove_rats(images)

        try:
          os.makedirs(image_output, exist_ok=True)
        except OSError as e:
            print(f'Error: {e}')

        output_path = os.path.join(image_output, f"{file_name}.jpg")
        print(output_path)
        cv2.imwrite(output_path, result)

# Imagens:

In [ ]:
def contour_properties(contour):
  """
  Computes various properties of a contour.

  Args:
      contour (numpy.ndarray): Input contour.

  Returns:
      tuple: Contour properties including area, circularity, radius, bounding box dimensions,
              aspect ratio, bounding box coordinates, and centroid coordinates.
  """
  import cv2
  import numpy as np

  # Calculate moments of the contour
  M = cv2.moments(contour)

  # Calculate area of the contour
  area = cv2.contourArea(contour)

  # Calculate perimeter of the contour
  perimeter = cv2.arcLength(contour, True)

  # Calculate bounding box dimensions and aspect ratio
  x, y, w, h = cv2.boundingRect(contour)
  aspect_ratio = max(float(w), h) / min(float(w), h) if min(w, h) > 0 else 0

  # Calculate minimum enclosing rectangle and circle
  rect = cv2.minAreaRect(contour)
  circle = cv2.minEnclosingCircle(contour)

  # Calculate rectangle coordinates and area
  box = cv2.boxPoints(rect)
  box = np.int0(box)
  rect_area = w * h

  # Calculate centroid using different methods
  center_x = int(M["m10"] / M["m00"]) if M["m00"] != 0 else 0
  center_y = int(M["m01"] / M["m00"]) if M["m00"] != 0 else 0

  # Calculate circularity of the contour
  circularity = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0

  # Calculate circle properties
  (_, _), radius = cv2.minEnclosingCircle(contour)
  center_circle = (int(center_x), int(center_y))

  # Calculate square properties
  cx_quad = int(x)
  cy_quad = int(y)

  return (area, circularity, radius, x, y, w, h, aspect_ratio,
          box, center_circle, center_x, center_y)

In [ ]:
def process_contours(frame_name, frame, contours, background):
    """
    Processes contours found in an image, categorizing them based on certain criteria.

    Args:
        frame_name (str): Name of the frame or image.
        frame (numpy.ndarray): Base image for drawing the contours.
        contours (list): List of contours found in the image.

    Returns:
        tuple: Tuple containing the processed image, object count, and dictionary of contour data.
    """
    import cv2

    object_count = 0
    contour_data = {}  # Dictionary to store data of identified objects

    previous_contour = None  # Variável para armazenar o contorno anteriormente escolhido
    previous_contour_area = 0  # Variável para armazenar a área do contorno anteriormente escolhido

    for contour in contours:
        # Approximate the contour with a sequence of line segments
        approx = cv2.approxPolyDP(contour, 0.01 * cv2.arcLength(contour, True), True)

        # Calculate contour properties
        area, circularity, radius, x, y, w, h, aspect_ratio, box, center_circle, cx_circle, cy_circle = contour_properties(contour)

        if background == 1:
            w += 20

        # Check if the contour matches a circular pattern
        if area > 2000 and 20 < radius < 100 and  x < 1000 and 150 < y < 400 and 80 < w < 160:
            # Limit the radius to a maximum value
            max_radius = 100
            radius = min(radius, max_radius)
            radius = int(radius)

            if radius < 80:
                radius += 30

            # Draw a circular contour
            cv2.circle(frame, center_circle, radius, (255, 0, 0), 2)

            # Draw the central point
            cv2.circle(frame, (cx_circle, cy_circle), 3, (255, 0, 0), -1)

            # Increment object count
            object_count += 1

            # Define the text to be written on the image
            text = f"Object{object_count}"

            # Coordinates where the text will be placed (top-left corner)
            coordinates = (cx_circle, cy_circle)

            # Specify the font and text size
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_size = 0.5
            color = (0, 0, 0)  # Text color in BGR (white in this example)

            # Add the text to the image
            image_with_text = cv2.putText(frame, text, coordinates, font, font_size, color, thickness=2)

            # Capture data
            contour_data[f"{frame_name}_{object_count}"] = [{
                "Form": "standard",
                "object": object_count,
                "Area": area,
                "Circularity": circularity,
                "Width": w,
                "Height": h,
                "X": cx_circle,
                "Y": cy_circle,
                "Radius": radius,
            }]

            # Verifica se o contorno atual contém o contorno anteriormente escolhido
            if previous_contour is not None and area > previous_contour_area:
                # Define o contorno atual como o contorno anteriormente escolhido
                previous_contour = contour
                previous_contour_area = area
            elif previous_contour is None:
                # Define o contorno atual como o contorno anteriormente escolhido
                previous_contour = contour
                previous_contour_area = area

    return frame, contour_data

In [ ]:
def find_contours(image):
    """
    Finds contours in the input image using the RETR_TREE mode and CHAIN_APPROX_SIMPLE method.

    Args:
        image (numpy.ndarray): Input image.

    Returns:
        list: List of contours found in the image.
    """
    import cv2

    # Find contours in the image using specified retrieval mode and approximation method
    contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    return contours

In [ ]:
def apply_dilation(image):
  """
  Applies dilation to the input image using a defined kernel.

  Args:
      image (numpy.ndarray): Input image.

  Returns:
      numpy.ndarray: Dilated image.
  """
  import cv2
  import numpy as np

  # Define the kernel for dilation
  kernel = np.ones((5, 5), np.uint8)

  # Perform dilation on the image
  dilated_image = cv2.dilate(image, kernel, iterations=3)

  return dilated_image

In [ ]:
def apply_threshold(image):
  """
  Applies thresholding to the input image using the Otsu method.

  Args:
      image (numpy.ndarray): Input image in grayscale format.

  Returns:
      numpy.ndarray: Thresholded image.
  """
  import cv2

  # Compute the gradients using Sobel operators
  sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
  sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

  # Calculate the magnitude of the gradient
  gradient_magnitude = cv2.magnitude(sobelx, sobely)

  # Normalize the gradient magnitude
  gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

  # Apply thresholding using the Otsu method
  _, gradient_threshold = cv2.threshold(gradient_magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

  return gradient_threshold

In [ ]:
def apply_filters(image, cam=0):
  """
  Applies filters to enhance the input image.

  Args:
      image (numpy.ndarray): Input image in grayscale format.
      cam (boolean): 0 means the field is light, 1 means the field is dark
  Returns:
      numpy.ndarray: Filtered image.
  """
  import cv2
  import numpy as np

  if cam:
    # Equalize the histogram of the input image
    equalized_image = image.copy()
  else:
    # Equalize the histogram of the input image
    equalized_image = cv2.equalizeHist(image)

  # Convert the equalized image to float32 format
  image32f = np.float32(equalized_image)

  # Compute mean and variance using a 3x3 kernel
  mu = cv2.blur(image32f, (3, 3))
  mu2 = cv2.blur(image32f * image32f, (3, 3))

  sigma = cv2.sqrt(mu2 - mu * mu)

  # Scale sigma for better visualization
  sigma = sigma * 10

  # Convert sigma to unsigned 8-bit integer
  sigma = sigma.astype("uint8")

  # Suppress edges by setting a border region to zero
  sigma[:20, :] = 0
  sigma[-20:, :] = 0
  sigma[:, :20] = 0
  sigma[:, -20:] = 0

  return sigma

In [ ]:
def process_frame(frame, background):
  """
  Processes a single frame from an image file.

  Args:
      frame (str): The path to the input image file.
      background (boolean): 0 means the field is light, 1 means the field is dark

  Returns:
      tuple: A tuple containing processed results and the image name.
  """
  import cv2
  import os

  # Read the image from the specified path
  image = cv2.imread(frame)

  # Extract the image name from the frame path
  image_name = os.path.basename(frame).split('.')[0]

  # Convert the image to grayscale
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Apply filters to the grayscale image
  filtered_image = apply_filters(gray_image, background)

  # Apply thresholding to the filtered image
  thresholded_image = apply_threshold(filtered_image)

  # Apply dilation to the thresholded image
  dilated_image = apply_dilation(thresholded_image)

  # Find contours in the dilated image
  contours = find_contours(dilated_image)

  # Process contours and get results
  results = process_contours(image_name, image, contours, background)

  # Return processed results and the image name
  return results, image_name

# Dataset:

In [ ]:
def create_dataset_image(dataset):
    """
    Creates a DataFrame from a dataset containing objects with attributes.

    Args:
        dataset (dict): Dictionary containing objects with attributes.

    Returns:
        pandas.DataFrame: DataFrame containing objects and their attributes.
    """
    import pandas as pd
    # Create an empty list to store objects
    object_list = []

    # Iterate through the dictionary
    for attribute, obj_list in dataset.items():
        # Iterate through the list of objects for the current attribute
        for obj in obj_list:
            # Check if obj is a dictionary
            if isinstance(obj, dict):
                # Add the object combined with the dictionary {'attribute': attr}
                object_list.append({**{'attribute': attribute}, **obj})

    # Create the DataFrame based on the object list
    df = pd.DataFrame(object_list)

    # Return the DataFrame if needed for further use
    return df

# Função Pilar:

In [ ]:
def run_system(image_pack, background=0):
  """
  Processes images in the specified image pack and generates results and data.

  Args:
      image_pack (str): Path to the image pack directory containing JPG images.
      background (boolean): 0 means the field is light, 1 means the field is dark
  Returns:
      None
  """
  import os
  import glob
  import cv2

  dados_gerais = {}

  frames_input = glob.glob(os.path.join(image_pack, '*.jpg'))

  for frame in frames_input:
      results, image_name = process_frame(frame, background)

      output_path = os.path.join(image_pack, "results")

      try:
          os.makedirs(output_path, exist_ok=True)
      except OSError as e:
          print(f'Error creating directory: {e}')

      output_image_path = os.path.join(output_path, f"{image_name}_completed.jpg")

      # Save the resulting image
      cv2.imwrite(output_image_path, results[0])
      cv2_imshow(results[0])

      # Call create_dataset_image function with the generated dataset
      dados_frame = create_dataset_image(results[1])

      # Update the overall dictionary with image data
      dados_gerais.update(results[1])

      output_data_path = os.path.join(output_path, f"{image_name}_data.csv")

      # Save the DataFrame to a CSV file
      dados_frame.to_csv(output_data_path, index=False)

  # Generate a dataset from the overall dictionary of all images
  dataset_geral = create_dataset_image(dados_gerais)

  output_all_data_path = os.path.join(image_pack, "results", "all_data.csv")

  # Save the DataFrame to a CSV file
  dataset_geral.to_csv(output_all_data_path, index=False)

# Função Explorar

# Teste da Biblioteca:

In [ ]:
import os
import glob
import numpy as np

import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/

Mounted at /content/drive/
/content/drive/MyDrive


In [ ]:
video_input = glob.glob('/content/drive/MyDrive/Lab eletrofisio ISD/COMPORTAMENTO/Padronização/Exp35-MRO_ contextos_dicas/Exp 35 - rodada 1/*.mp4')
raw_image_output = '/content/drive/MyDrive/PPGNeuro/Projeto - Algoritmos/area-Antares/test-exp35'

In [ ]:
#extract_frames(video_input, raw_image_output)

In [ ]:
run_system(raw_image_output, 0)